In [1]:
import yaml
from collections import defaultdict
import pprint
pp = pprint.PrettyPrinter(indent=4)
# %pprint # turn on /off

In [4]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf_all = yaml.safe_load(config_file)

In [5]:
conf_all

{'connection': {'host': '172.22.0.1',
  'db': 'dv2',
  'port': 5432,
  'user': 'postgres',
  'pw': 'postgres',
  'schemas': {'stage': 'stage', 'edwh': 'vault'}},
 'rv': {'hubs': {'hub_tax_bundle': {'hk': 'hk_tax_bundle',
    'bk': 'tax_bundle_id',
    'overwrite': False,
    'src': {'v_report_taxation_bundle': {'bk': 'tax_bundle_id',
      'bkeycode': 'default',
      'tenant': 'default'},
     'v_report_ziffer': {'bk': 'tax_bundle_id'}}},
   'hub_ziffer': {'hk': 'hk_ziffer',
    'bk': 'ziffer',
    'overwrite': False,
    'src': {'v_report_dict_ziffer': {'bk': 'ziffer'}}}},
  'links': {'lnk_taxbundle_ziffer': {'hk': 'hk_l_taxbundle_ziffer',
    'overwrite': False,
    'src': {'v_report_ziffer': {'bkeycode': 'default',
      'tenant': 'default',
      'bks': {'tax_bundle_id': 'hk_tax_bundle', 'ziffer': 'hk_ziffer'},
      'cks': ['snapshot_version']},
     'v_report_taxation_bundle': {'bkeycode': 'default',
      'tenant': 'default',
      'bks': {'tax_bundle_id': 'hk_tax_bundle', 'zif

In [457]:
def defaultify(d):
    if isinstance(d, dict):
        return defaultdict(lambda: None, {k: defaultify(v) for k, v in d.items()})
    elif isinstance(d, list):
        return [defaultify(e) for e in d]
    else:
        return d

In [58]:
dconf=defaultify(conf_all)

# hubs


## loader

In [79]:
sql= open('./dags/sql/hub_loader.sql','r').read()

In [66]:
sql

"with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (\n\tselect md5(concat_ws('|',\n\t\t\t   {tenant},{bkeycode},\n\t\t\t   trim(src.{bk_src}::varchar(256),\n\t)) \n\t, trim(src.{bk_src}::varchar(256))\n\t, current_timestamp \n\t, coalesce({appts::timestamp},current_timestamp))\n\t, {rec_src}\n\t, {tenant}\n\t, {bkeycode}\n\t, {taskid}\t\n\tfrom {schema_src}.{table_src} src\n)\ninsert into {schema_tgt}.{table_tgt} (\n\t\t{hk_tgt}\n\t  , {bk_tgt}\n\t  , dv_loadts\n\t  , dv_appts\n\t  , dv_recsrc\n\t  , dv_tenant\n\t  , dv_bkeycode\n\t  , dv_taskid\n\t)\nselect distinct \n\thk\n  , bk\n  , dv_loadts\n  , dv_appts\n  , dv_recsrc\n  , dv_tenant\n  , dv_bkeycode\n  , dv_taskid\nfrom cte\nwhere not exists \n(\n\tselect 1 from {schema_tgt}.{table_tgt} tgt\n\twhere cte.hk = tgt.{hk_tgt}\n);"

In [80]:
conf=dconf['rv']

appts=None
sql_concat=''
hub='hub_tax_bundle'
schema_stage='stage'
schema_edwh='vault'

for cur_src,cur_conf in conf['hubs'][hub]['src'].items():

    params={'tenant':cur_conf['tenant'] if cur_conf['tenant'] else "default",
            'bkeycode':cur_conf['bkeycode'] if cur_conf['bkeycode'] else "default",
            'taskid': '1234',
            'appts':'appts::timestamp' if appts else 'current_timestamp',
            'rec_src':cur_src,
            'bk_src':cur_conf['bk'],
            'schema_src':schema_stage,
            'table_src':cur_src,
            'bk_tgt':conf['hubs'][hub]['bk'],
            'hk_tgt':conf['hubs'][hub]['hk'],
            'schema_tgt':schema_edwh,
            'table_tgt':hub
    }

    #print(sql.format(**params))
    sql_concat += '\n'+sql.format(**params)
    
print(sql_concat)    


with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (
	select md5(concat_ws('|',
			   default,default,
			   trim(src.tax_bundle_id::varchar(256),
	)) 
	, trim(src.tax_bundle_id::varchar(256))
	, current_timestamp 
	, coalesce(current_timestamp::timestamp,current_timestamp)
	, v_report_taxation_bundle
	, "default"
	, "default"
	, 1234	
	from stage.v_report_taxation_bundle src
)
insert into vault.hub_tax_bundle (
		hk_tax_bundle
	  , tax_bundle_id
	  , dv_loadts
	  , dv_appts
	  , dv_recsrc
	  , dv_tenant
	  , dv_bkeycode
	  , dv_taskid
	)
select distinct 
	hk
  , bk
  , dv_loadts
  , dv_appts
  , dv_recsrc
  , dv_tenant
  , dv_bkeycode
  , dv_taskid
from cte
where not exists 
(
	select 1 from vault.hub_tax_bundle tgt
	where cte.hk = tgt.hk_tax_bundle
);
with cte (hk,bk,dv_loadts,dv_appts,dv_recsrc,dv_tenant,dv_bkeycode,dv_taskid) as (
	select md5(concat_ws('|',
			   default,default,
			   trim(src.tax_bundle_id::varchar(256),
	)) 
	, trim(src.tax_bundle_

# links

In [476]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf = yaml.safe_load(config_file)

In [477]:
dconf=defaultify(conf)

## link creator

In [478]:
conf=conf['rv']

In [481]:
link='lnk_taxbundle_ziffer'

In [482]:
# collect bks -> hks mappping from all sources
bk_list=list()
hk_list=list()

for cur_src, cur_conf in conf['links'][link]['src'].items():
    
    #link_bks_dict={**link_bks_dict,**cur_conf['bks']} # merge dicts
    #print(link_bks_dict)
    bk_list +=[x for x in cur_conf['bks'].keys()]
    hk_list +=[x for x in cur_conf['bks'].values()] # if x != None]
    
    

In [483]:
# das brauche ich
# ich möchte, dass der user in conf später irgendwann auch die reihenfolge seiner quellen und die bks pro quelle ändern darf.
# deswegen brauche ich diese sortierten dicts

bk_hk_dict = dict(zip(bk_list,hk_list))
bk_hk_dict_sorted=dict(sorted(bk_hk_dict.items()))
hk_bk_dict_sorted=dict(zip([x for x in bk_hk_dict_sorted.values()],[x for x in bk_hk_dict_sorted.keys()]))

### hk_list_create

In [490]:
hk_unique = [ f'{x} varchar(256) NOT NULL' for x in hk_bk_dict_sorted.keys() if x != None]
hk_list_create_str = ','.join(hk_unique)
print(hk_list_create_str)

hk_tax_bundle varchar(256) NOT NULL,hk_ziffer varchar(256) NOT NULL


### bk_list_create

In [491]:
bk_list_create = [ f'{x} varchar(256) NOT NULL' for x in hk_bk_dict_sorted.values()]
bk_list_create_str = ','.join(bk_list_create)
print(bk_list_create_str)

dossier_uuid varchar(256) NOT NULL,tax_bundle_id varchar(256) NOT NULL,ziffer varchar(256) NOT NULL


### format

In [495]:
sql= open('./dags/sql/link_creator.sql','r').read()

In [496]:
# schema_edwh = Variable.get('SCHEMA_EDWH')
schema_edwh = 'vault'

In [497]:
params = {
    'schema':schema_edwh,
    'link':link,
    'hk':conf['links'][link]['hk'],
    'bk_list_create':bk_list_create_str,
    'hk_list_create':hk_list_create_str
}

In [498]:
print(sql.format(**params))

CREATE TABLE vault.lnk_taxbundle_ziffer (
	hk_l_taxbundle_ziffer varchar(256) NOT NULL,
	hk_tax_bundle varchar(256) NOT NULL,hk_ziffer varchar(256) NOT NULL,
	dossier_uuid varchar(256) NOT NULL,tax_bundle_id varchar(256) NOT NULL,ziffer varchar(256) NOT NULL,
	-- technical fields
	DV_LOADTS timestamp NULL,
	DV_APPTS timestamp NULL,
	DV_RECSRC varchar(256) NULL,
	DV_TENANT varchar(256) NULL,
	DV_BKEYCODE varchar(256) NULL,
	DV_TASKID varchar(256) NULL,
    CONSTRAINT lnk_taxbundle_ziffer_pkey PRIMARY KEY (hk_l_taxbundle_ziffer)
);


## link loader

In [502]:
sql= open('./dags/sql/link_loader.sql','r').read()

In [503]:
link='lnk_taxbundle_ziffer'

In [504]:
with open('dags/configs/conf.yaml', "r") as config_file:
    conf = yaml.safe_load(config_file)
    
dconf=defaultify(conf_all)
conf=dconf['rv']

In [505]:
# collect bks -> hks mappping from all sources
bk_list=list() # total
hk_list=list() # total

bk_hk_obj={} # per source
for cur_src, cur_conf in conf['links'][link]['src'].items():
    
    #link_bks_dict={**link_bks_dict,**cur_conf['bks']} # merge dicts
    #print(link_bks_dict)
    bk_list +=[x for x in cur_conf['bks'].keys()]
    hk_list +=[x for x in cur_conf['bks'].values()]
    bk_hk_obj[cur_src]=cur_conf['bks']

In [506]:
# das brauche ich
# ich möchte, dass der user in conf später irgendwann auch die reihenfolge seiner quellen und die bks pro quelle ändern darf.
# deswegen brauche ich diese sortierten dicts

bk_hk_dict = dict(zip(bk_list,hk_list))
bk_hk_dict_sorted=dict(sorted(bk_hk_dict.items()))
hk_bk_dict_sorted=dict(zip([x for x in bk_hk_dict_sorted.values()],[x for x in bk_hk_dict_sorted.keys()]))


In [507]:
bk_hk_dict_sorted

{'tax_bundle_id': 'hk_tax_bundle', 'ziffer': 'hk_ziffer'}

In [508]:
hk_bk_dict_sorted

{'hk_tax_bundle': 'tax_bundle_id', 'hk_ziffer': 'ziffer'}

In [509]:
# das brauche ich (CTE & INSERT)
hk_unique = [ x for x in hk_bk_dict_sorted.keys() if x != None]
hk_unique

['hk_tax_bundle', 'hk_ziffer']

### hk_list_keys

In [510]:
# das brauche ich (CTE)
hk_list_keys_str = ','.join(hk_unique)
hk_list_keys_str

'hk_tax_bundle,hk_ziffer'

### bk_list_keys

In [511]:
# in den quellen können die bks anders heißen auch wenn sie auf denselben hk mappen
# nehme einfach den ersten treffer

In [512]:
bk_list_keys = list(hk_bk_dict_sorted.values())
bk_list_keys_str = ','.join(bk_list_keys)
print(bk_list_keys_str)

tax_bundle_id,ziffer


### hk_list_vals, bk_list_vals, concat_str

In [521]:
#tenant= 'default'
#bkeycode = 'default'

# for cur_src, cur_conf in ...
cur_src = 'v_report_ziffer'
#cur_src = 'v_report_taxation_bundle'

cur_obj=bk_hk_obj[cur_src]
cur_obj_inv=dict(zip(cur_obj.values(),cur_obj.keys()))

# hk_list für INSERT
hk_list_vals = list(map(lambda x: 
         f"md5(CONCAT_WS('|',\'{tenant}\',\'{bkeycode}\', trim({x}::varchar(256))))",
         [cur_obj_inv[x] for x in hk_unique]))

hk_list_vals_str=',\n'.join(hk_list_vals)
print(hk_list_vals_str)

print("\n")


# bk_list für INSERT
bk_list_vals = [cur_obj_inv[y] if y in cur_obj_inv.keys() else '-1' for y in hk_bk_dict_sorted.keys()]
bk_list_vals_str="::varchar(256),\n".join(bk_list_vals) + "::varchar(256)"
print(bk_list_vals_str)

print("\n")

concat_list = [cur_obj_inv[y] for y in hk_bk_dict_sorted.keys() if y in cur_obj_inv.keys() and y]
concat_list_str =f"concat_ws('|',\'{tenant}\',\'{bkeycode}\'," +','.join(list(map(lambda x:f'{x}::varchar(256)',concat_list)))+')'
print(concat_list_str)

md5(CONCAT_WS('|','default','default', trim(tax_bundle_id::varchar(256)))),
md5(CONCAT_WS('|','default','default', trim(ziffer::varchar(256))))


tax_bundle_id::varchar(256),
ziffer::varchar(256)


concat_ws('|','default','default',tax_bundle_id::varchar(256),ziffer::varchar(256))


### Todo: 
- Was ist, wenn es mehrere bks gibt die auf "NULL" zeigen ???

In [133]:
bk_list_vals_str = ','

In [121]:
bk_list2

'tax_bundle_id,ziffer'

### putting together

In [462]:
sql= open('./dags/sql/link_loader.sql','r').read()

In [522]:
appts=None
sql_concat=''
link='lnk_taxbundle_ziffer'
schema_stage='stage'
schema_edwh='vault'

for cur_src,cur_conf in conf['links'][link]['src'].items():

    cur_obj=bk_hk_obj[cur_src]
    cur_obj_inv=dict(zip(cur_obj.values(),cur_obj.keys()))

    # hk_list für INSERT
    hk_list_vals = list(map(lambda x: 
             f"md5(CONCAT_WS('|',\'{tenant}\',\'{bkeycode}\', trim({x}::varchar(256))))",
             [cur_obj_inv[x] for x in hk_unique]))

    hk_list_vals_str=',\n'.join(hk_list_vals)

    # bk_list für INSERT
    bk_list_vals = [cur_obj_inv[y] if y in cur_obj_inv.keys() else '-1' for y in hk_bk_dict_sorted.keys()]
    bk_list_vals_str="::varchar(256),\n".join(bk_list_vals) + "::varchar(256)"

    concat_list = [cur_obj_inv[y] for y in hk_bk_dict_sorted.keys() if y in cur_obj_inv.keys() and y]
    concat_list_str =f"\'{tenant}\',\'{bkeycode}\'," +','.join(list(map(lambda x:f'concat({x}::varchar(256))',concat_list)))

    
    params={'tenant':cur_conf['tenant'] if cur_conf['tenant'] else "default",
            'bkeycode':cur_conf['bkeycode'] if cur_conf['bkeycode'] else "default",
            'taskid': '1234',
            'appts':'appts::timestamp' if appts else 'current_timestamp',
            'rec_src':cur_src,
            #            
            'schema_src':schema_stage,
            'table_src':cur_src,
            'schema_tgt':schema_edwh,
            'table_tgt':link,
            #
            'hk_list_keys':hk_list_keys_str,
            'bk_list_keys':bk_list_keys_str,
            'hk_list_vals':hk_list_vals_str,
            'bk_list_vals':bk_list_vals_str, 
            'concat_str':concat_list_str,
            'hk_tgt':conf['links'][link]['hk']
    }

    #print(sql.format(**params))
    sql_concat += '\n'+sql.format(**params)
    
print(sql_concat)   


with cte (hk,
		 hk_tax_bundle,hk_ziffer,
		 tax_bundle_id,ziffer,
		 DV_LOADTS, 
		 DV_APPTS, 
		 DV_RECSRC, 
		 DV_TENANT, 
		 DV_BKEYCODE, 
		 DV_TASKID
	) as (
	select md5('default','default',concat(tax_bundle_id::varchar(256)),concat(ziffer::varchar(256))) 
	, md5(CONCAT_WS('|','default','default', trim(tax_bundle_id::varchar(256)))),
md5(CONCAT_WS('|','default','default', trim(ziffer::varchar(256))))
	, tax_bundle_id::varchar(256),
ziffer::varchar(256)
	,  current_timestamp 
	, current_timestamp
	, 'v_report_ziffer'
	, 'default'
	, 'default'
	, '1234	'
	from stage.v_report_ziffer src
)
insert into vault.lnk_taxbundle_ziffer (
	hk_l_taxbundle_ziffer,
    hk_tax_bundle,hk_ziffer,
    tax_bundle_id,ziffer,
	DV_LOADTS, 
	DV_APPTS, 
	DV_RECSRC, 
	DV_TENANT, 
	DV_BKEYCODE, 
	DV_TASKID	
)
select distinct hk,
    hk_tax_bundle,hk_ziffer,
    tax_bundle_id,ziffer,
	DV_LOADTS, 
	DV_APPTS, 
	DV_RECSRC, 
	DV_TENANT, 
	DV_BKEYCODE, 
	DV_TASKID	
from cte
where not exists 
(
	select 1 from vau